In [15]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
df = pd.read_csv('amazon.csv')
df["label"].value_counts()   # Unbalanced dataset

label
1    15230
0     4766
Name: count, dtype: int64

In [18]:
X = df['Text']
y = df['label']   # 1 for positive, 0 for negative
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
pipe = Pipeline([('vectorizer', CountVectorizer()), ('model', LogisticRegression(max_iter=1000))])
pipe.fit(X_train, y_train)  
print("Pipe score: ",pipe.score(X_test, y_test))
y_pred = pipe.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balaced accuracy: ", accuracy)


Pipe score:  0.89675
Balaced accuracy:  0.8433865047995482


In [30]:
# Testing for arbitray text
text_positive = ["This product is unbelievably good"]
text_negative = ["This product is unbelievably bad"]  # For some reason this is classified as positive
prediction_positive = pipe.predict(text_positive)
prediction_negative = pipe.predict(text_negative)
print("Prediction positive: ", prediction_positive)
print("Prediction negative: ", prediction_negative)

Prediction positive:  [1]
Prediction negative:  [1]
